In [1]:
from PIL import Image, ImageOps, ImageFont
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
from ipywidgets import interactive, HBox, VBox
from ascii_art_conversion_1x1 import img2char_arr_1x1
from ascii_art_conversion_1x2 import img2char_arr_1x2
from img_processing import preprocess_img, DITHER_MODES
from braille_art import img2braille_arr, get_braille_chars
from generate_ascii_palette import *

In [2]:
ASCII_SET = get_asciis()
BRAILLE_SET = get_braille_chars()
ASCII_AND_BRAILLE_SET = ASCII_SET + BRAILLE_SET

In [3]:
FONT = ImageFont.truetype("fonts/CascadiaMono.ttf", 16)
IMG_PATH = "imgs/irad_grad.bmp"

In [4]:
orig_img = Image.open(IMG_PATH).convert("L")
#orig_img = ImageOps.invert(orig_img)

print(orig_img.size)

(512, 512)


In [5]:
def i_preprocess_img(scale_factor, contrast, brightness, quantize_colors, eq, dither):
    proc_img = preprocess_img(orig_img, 0.1, contrast, brightness, eq, quantize_colors, dither)
    plt.imshow(proc_img, cmap='gray', vmin=0, vmax=255, interpolation='none')

    plt.figure(figsize=(9, 3))
    plt.xticks([x for x in range(0, len(proc_img.histogram()), 25)])
    plt.bar([x for x in range(0, len(proc_img.histogram()))], proc_img.histogram())
    plt.grid()
    plt.show()

    return proc_img

interactive_preprocess = interactive(i_preprocess_img,
            scale_factor=(0.01, 1, 0.01),
            contrast=(0, 2, 0.01),
            brightness=(0, 2, 0.01),
            eq=(0, 1, 0.01),
            quantize_colors=(1, 256, 1),
            dither=DITHER_MODES)

controls = VBox(interactive_preprocess.children[:-1])
display(HBox((controls, interactive_preprocess.children[-1])))

In [6]:
proc_img = interactive_preprocess.result

In [7]:
palette, palette_br = generate_1_1_palette(ASCII_SET, FONT, 12, normalize=True)
ascii_arr = img2char_arr_1x1(proc_img, palette, palette_br, dither=DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

   ....,::''"_!!-!^~^"!"~^";!;^;!""-`-,_'`'.'...   
 ..,._``!_-!-~"~"""""\""+"*"*""""""""~"^!",;_'`._..
._,_,"!'-^^"""">"/?>?"l?>L?*>=L?L"*""""""^"!`_:'`..
.`,_^,~;^"""<"\=*<>/<Y??+>\/+/L=+\\?L+"/""^!";;____
'`~:!^!"""/+\<?//?\rTLTzi)J))7Yc+J>?|\\"+""""~~~~:_
'^;;;"""|L">\L=u)J(jn}iXeSSSj3JrJ(rjL<|=L/"?""!~'!:
';-;""L"/=<*)jvT)IstfyaXy[2{I}yS{oTY(Tr?+>+"""""~-'
!~";"<"L=?ci(1)]XaS5{GbUdbmGphE}XFIe(rY(\*+?>\"""~^
!-~"<"|L*J+1xrsee5{96q46%P&6hwb4k6sIC2rzT=r>|">""--
;;"">*\=T\i1[]ob}6dk%%&%80g0RMkm4qdX2e[jzY+|LL"<""^
;""/"=>>TvvVC3yd6GRM5g&BNBN$RM80bUKdV3CtY)J=||+""""
-""<?L\v\uFz[53OhH%MDW$QB@$WNBBR%PqKd[aaxic=?=L"L"~
!""<"+>*j(7a[ewGG#d#BNQ@@@@@WWQD#M4k9[oI}cjn/*L""""
~""\*>=j)zvoZkZkPEg#%$BQB@Q$N$%&DOEPZp3[7lz+>*L"/""
;"""="</x(vI[2a6dk8g0&NWWNBNBRD8AhbPo3Zsxr7|L="\"""
;""="+|><7)1eSVq}466wgM&M#8%8RgPHG5esSeYv1<<\?+""";
^!""L"\\=uLvJVZIIPyKUwhU%5&E5Uh96stf23ZJc/i/L""*"";
!;"""="L>=TxuY(ZZ22m3KE5kwkpqb[tyV2SFxjz+/||"/""""^
`""^""\"+L/**n11jaZ3Z{X]}C[F}SVsaXSTnz>\//\"<"""^"_
'-:""""|"=">

In [8]:
palette, palette_br = generate_1_2_palette(ASCII_SET, FONT, (8,8), normalize=True)
ascii_arr = img2char_arr_1x2(proc_img, palette, palette_br, dither=DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

   .:--:-:-;-!;~!~~!~!!!!!!!!!!!!~!~;!~;-:-::.: .  
 --::----;~!~~!~!>!+=*++>/*==/*\\<!!!!!;~;--:-.-.- 
...-:-:!;!~!!=\//>|*|+++>==>=>=*+>>|\\!!!~~;--:-.: 
.:--;-~~!!=\/*+/>*n/txY{)U)I(c)j>//|==>|<!~~!;-:---
:--;!~!!+\!/=>J))jtIj}3][}IV2JVr2C(J)/*|=<\!!!~;--:
-;-~~~+<*>/)iln]3VIUtkIkhkX{k213C2Vilx(/=+|*<!~!;-:
-;!!!+!+**c>tj12IkS6HkOAkdE6Eb6HOU{I[Iilc>|//!/!~!;
;!!!/>=/)j)3t1tkhk6A6KO&b&AQ4dbEbGbh1}{cYcl|>>!>~!;
;~<!+===nl132k5kKKAW#$QQN&B&WN0WOOddSk}}}xY==+/!/~!
~!!>=>=Ii[]Uk5Kd4QdBQBBBBB@WBBBQbBG64kS}}})cl++>~!~
!<<=>=J>t1[{kObOW%BBBB@@@@@@B@BBBBQKEK6}k{3j=|=|<<~
~!+!*j=2[tVOSd6WABBB@@@@@@@@@@B@BBQbOAAk}{Vx)>**!/~
~\\*>*tj[[IbAdGQBBB@B@@@@@@@@@@BBBQNA6OktUIn)==>|~!
~!|+>/n=}{]kKbbW8BBB@@@@@@@@@@@BBBBAWOd1Z3](cl>>\\~
~<!||>cY1V}kH4G6Q$BBB@B@@@@@@B@BBB#4EGdZCUtYr*>+<!!
~~=\*/=)x}V1GZbE4B$BBBB@B@BBBBBBBWDbOdh}t[[r/>=/~<~
~!!*\++lJ(t{tdhGKdRBBBBBBBBBBBBDE64OGX}X2}+x>**<~~~
!!!\!|++n)Ut]VXkZOdODd%R$NB0NW%KEOOX}X2CCY=>|=\~\!!
;~!\\\=**|cY[C}{kh56OOE$4DGDOKOAhhV5C1{(xl+l><~<!!~
-;~!~!><=>+c

In [9]:
ascii_arr = img2braille_arr(proc_img, dither=DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

⠀⠀⠠⢀⠂⠔⡐⡐⢔⢐⢅⠢⡊⢔⡐⢅⠢⡂⡂⡂⡂⠄⠂⡀⠀
⠀⠌⡐⠄⢅⠕⡐⡅⡕⡌⡆⡇⡣⡣⡪⢪⢨⢢⠱⡐⢌⠌⡐⠠⢁
⠈⠔⡨⠨⡢⡑⡅⡇⣕⢕⡕⡵⡱⣕⢵⡱⡕⡕⡕⠥⡑⡌⢌⠌⢄
⢈⠢⡊⢜⠔⡕⡕⡵⡕⡧⣫⣞⣝⢮⣺⢜⡮⡪⡎⡇⡇⡕⢅⠪⡐
⢐⢅⠪⡊⡎⡮⡺⡪⡯⣞⣗⡷⣽⣽⣺⣳⢽⣹⣪⢺⡸⡸⡨⡊⠔
⠰⡐⢕⢕⢕⢵⢝⡽⣝⡾⣞⣿⢷⣿⣞⣷⡻⡮⡮⡣⡇⡇⡎⡜⡨
⠨⠪⡸⢸⢸⢪⡳⣝⡾⣽⣻⡿⣿⣿⣽⡾⣯⣻⡪⣏⢞⢜⢜⠔⢌
⠨⡱⡘⡜⡜⣕⢝⡞⣞⣽⣳⢿⣻⣽⢾⢯⣗⢗⣝⢮⢪⢣⢱⢑⠅
⠨⢂⢪⠸⡸⣘⢮⢺⢕⢷⢽⢽⢽⡺⡯⣳⡳⣝⢮⢺⢸⢸⢐⢅⠊
⢈⠢⠡⡱⡘⡔⡕⡵⡹⡱⣝⢮⡳⣝⢝⢮⢺⡸⡸⡸⡘⢔⠅⡢⠡
⢀⠊⠌⠔⢌⠢⡣⢱⢱⢹⢸⢸⢸⢸⢱⢱⢱⢱⢑⢅⠣⡑⠌⠄⠅
⠀⠄⠡⠑⡐⠡⡊⢌⠆⢕⢜⠸⡘⢜⠸⡘⢌⠢⡑⠔⡡⠂⠅⠌⠠


In [10]:
from ascii_art_conversion_mxn import quantize_grayscale_mxn

brightness_shape = (2,4)
char_to_brightness_map = generate_brightness_map(ASCII_SET, FONT, brightness_shape, normalize=True)
ascii_arr = quantize_grayscale_mxn(proc_img, char_to_brightness_map, (brightness_shape[1], brightness_shape[0]), DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

.::;!||||}}}}}}}}|||;;::.
:;|||}}[[[[[[[[[[[[[|||;:
;||[[[[[[Q[Q[Q[Q[[[[[[[|;
||[[[[[Q[QQQQQQQQ[E[[[[[|
|[[[[[QQQQ@@@@@@QQ[E[[[[[
[[[[[QQQ$@@@@@@@@$$[E[[[[
}}[[[Q$$$$$@@@@@$$$[E[[[[
[[[[[Q[$$$$$$$$$$$[E[[[[[
||[[[[[$[$$$$$$$$[E[[[[[|
||}[[[[[[[$[$[E[E[[[[[[[|
!|||[[[[[[[[[[[[[[[[[[||!
::!|||[[[[[[[[[[[[[[||!::
